<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_copy_archive_to_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Copy archived contracts using GetAccept *API*

Functions to copy selected or all archived documents/contracts from one entity to one or many subentities including name, type, value, tags, sign date, contract dates and company information.

__** NOTE ***__
- Attachments can not be moved between entities and has to be kept in original entity

In [ ]:
# import dependencies
import requests
import os
import io
import json
import copy
import mimetypes
import ipywidgets as widgets
from google.colab import files
from PIL import Image

#Constants 
BASE_URL = "https://api.getaccept.com/v1"
INT_BASE_URL = "https://int.getaccept.com"
destination_entities = []
SOURCE_ENTITY_ID = ""
last_header = ""
#@title ↓↓ Click here to start
#@markdown This step might take a few seconds to run. <br>
#@markdown Then use __shift+enter__ key or click ► left of each step to go through the flow

In [ ]:
email_widget = widgets.Text(
    value="",
    placeholder="Enter login email",
    description="Email:",
    disabled=False
)
password_widget = widgets.Password(
    value="",
    placeholder="Enter password",
    description="Password:",
    disabled=False
)
#@markdown Use the form below to fill in login details to your entity in GetAccept and then run next cell to login
widgets.VBox([email_widget, password_widget])

In [ ]:
#@markdown Login and store API token
if email_widget.value and password_widget.value:
  payload = { "email": email_widget.value, "password": password_widget.value}
  if SOURCE_ENTITY_ID != "":
    payload["entity_id"] = SOURCE_ENTITY_ID
  response = requests.post(BASE_URL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASE_URL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  SOURCE_ENTITY_ID = user_data["user"]["entity_id"]
else:
  raise TypeError("Could not login, missing email or password!")

In [ ]:
#@markdown Select the source entity you would like to get archived documents from. When you're done, run the next cell
source_entity_list = list(map(lambda x: (x["name"],x["id"]), user_data["entities"]))
source_entity_picker = widgets.Select(
    options=sorted(source_entity_list),
    value=SOURCE_ENTITY_ID,
)
source_entity_picker

In [ ]:
#@markdown Verifying entity token of source...
if source_entity_picker.value != SOURCE_ENTITY_ID:
  # Switch entity
  response = requests.get(BASE_URL+"/refresh/"+source_entity_picker.value, headers=source_auth_headers)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  SOURCE_ENTITY_ID = source_entity_picker.value
print("Authenticated to source entity \"%s\"" % source_entity_picker.label)

In [ ]:
#@markdown Select archive folder id:<br>
#@markdown Example from url: ...app.getaccept.com/contract-management/**r2j1hqkyx**<br>
#@markdown enter **r2j1hqkyx** - Leave empty for root folder<br>
contract_directory = ""
contract_directory_widget = widgets.Text(
    value=contract_directory,
    placeholder="Enter archive directory/path",
    description="Folder:",
    disabled=False
)
contract_directory_widget

In [ ]:
#@markdown Search for contracts:<br>
#@markdown *Enter a search term or leave empty to list all*<br>
search_directory = ""
search_directory_widget = widgets.Text(
    value=search_directory,
    placeholder="Enter contract search term",
    description="Search:",
    disabled=False
)
search_directory_widget

In [ ]:
#@markdown Select source documents(s) to copy:<br>
#@markdown _Hold down __Command__ to select multiple vales_<br>
#@markdown Select the first option in the list to copy all documents<br>
payload = {"operationName":"ContractManagementDocuments","variables":{"folderId":contract_directory_widget.value,"showInAllFolders":False,"search":[search_directory_widget.value],"offset":0,"limit":0,"tags":[],"users":[],"value":[0,10E10],"contractLength":[0,15],"contractEndDate":[None,None],"documentTypes":["sales","hr","introduction","other","document"],"signDate":["1970-01-01T00:00:00.000Z",None],"sortBy":"signDate","sortOrder":"DESC","withoutTags":False},"query":"query ContractManagementDocuments($search: [String!], $offset: Int, $limit: Int, $tags: [String!], $folderId: String, $users: [String!], $value: [Float!], $contractLength: [Int!], $contractEndDate: [DateTime], $documentTypes: [DocumentType!], $signDate: [DateTime], $sortBy: String, $sortOrder: String, $showInAllFolders: Boolean, $withoutTags: Boolean) {\n  contractManagement(\n    search: $search\n    offset: $offset\n    limit: $limit\n    tags: $tags\n    folderId: $folderId\n    users: $users\n    valueFloat: $value\n    contractLength: $contractLength\n    contractEndDate: $contractEndDate\n    documentTypes: $documentTypes\n    signDate: $signDate\n    sortBy: $sortBy\n    sortOrder: $sortOrder\n    showInAllFolders: $showInAllFolders\n    withoutTags: $withoutTags\n  ) {\n    name\n    id\n    type\n    isSigned\n    recipients {\n      firstName\n      lastName\n      email\n      __typename\n    }\n    user {\n      id\n      fullName\n      firstName\n      lastName\n      email\n      __typename\n    }\n    companyName\n    value\n    contractStartDate\n    contractEndDate\n    signDate\n    tags {\n      title\n      status\n      id\n      __typename\n    }\n    __typename\n  }\n}"}
result = requests.post(INT_BASE_URL+"/graphql?op=ContractManagementDocuments", json=payload, headers=source_auth_headers)
contract_result = result.json()
if contract_result.get("errors"):
  print(payload)
  print(contract_result)
  raise TypeError("Could not find any contracts!")
else:
  souce_contracts_filtered = list(filter(lambda x: x["name"] != "", contract_result["data"]["contractManagement"]))
  source_contract_list = list(map(lambda x: (x["name"],x["id"]), souce_contracts_filtered))
source_contract_list.insert(0, ('-- COPY ALL CONTRACTS --',''))
source_contract_widget = widgets.SelectMultiple(
    options=source_contract_list,
    value=['']
)
source_contract_widget

In [ ]:
#@markdown Select destination entities to copy contracts to:<br>
#@markdown _Hold down __Command__ to select multiple vales_
destination_entity_list = list(map(lambda x: (x["name"],x["id"]), list(filter(lambda x: x["id"] != SOURCE_ENTITY_ID, user_data["entities"]))))
destination_picker = widgets.SelectMultiple(
    options=sorted(destination_entity_list),
    rows=10
)
destination_picker

In [ ]:
#@markdown Get auth tokens for destination entities...
if len(destination_picker.value) > 0:
  if last_header == "":
    last_header = source_auth_headers
  destination_entities = []
  for i,entity_id in enumerate(destination_picker.value):
    response = requests.get(BASE_URL+"/refresh/"+entity_id, headers=last_header)
    data = response.json()
    if "access_token" in data:
      last_header = { "Authorization": "bearer " + data["access_token"]}
      destination_entities.append({
        "id": entity_id,
        "name": destination_picker.label[i],
        "header": last_header
      })
      print("Authenticated to destination entity \"%s\"" % destination_picker.label[i])
    else:
      raise TypeError(data["errors"], "Could not authenicate to entity " + destination_picker.label[i])
else:
  raise TypeError("No entities selected")

In [ ]:
#@markdown Preparing archive functions
def get_source_document(document_data, direct=False):
  # Download Source documents...
  result = requests.get(BASE_URL+"/documents/"+document_data["id"]+"/download" + ("?direct=true" if direct else ""), headers=source_auth_headers)
  if direct:
    return result.content
  else:
    document_result = result.json()
    if document_result.get("errors"):
      print(payload)
      print(document_result)
      raise ValueError(document_result["errors"][0]["message"])
    else:
      return document_result["document_url"]


def get_document_data(d, val):
    return next(item for item in d["data"]["contractManagement"] if item['id'] == val)

def upload_destination_document(source_document, source_document_data, destination):
      # Create new template with same name
      payload = {"name":source_document_data["name"], "type":source_document_data["type"], "value":source_document_data["value"] }
      files = None
      if type(source_document) == "str":
        payload["file_url"] = source_document
      else:
        files = {"file": source_document} 
      if source_document_data["signDate"]:
        payload["sign_date"] = source_document_data["signDate"]
      else:
        print("Skipping document \"%s\" - Missing sign-date" % source_document_data["name"]) 
        return
      if source_document_data["tags"]:
        tags = source_document_data["tags"]
        tags = ",".join([d['title'] for d in tags])
        payload["tags"] = tags
      if source_document_data["companyName"]:
        payload["company_name"] = source_document_data["companyName"]
      if source_document_data["contractStartDate"]:
        payload["contract_start_date"] = source_document_data["contractStartDate"]
      if source_document_data["contractEndDate"]:
        payload["contract_end_date"] = source_document_data["contractEndDate"]

      result = requests.post(BASE_URL+"/upload/archive", payload, files=files, headers=destination["header"])
      upload_result = result.json()
      if upload_result.get("message"):
        print(payload)
        print(upload_result)
        raise ValueError(upload_result["message"])
      else:
        return upload_result["document"]["id"]
if source_contract_widget.value[0] == '':
  count = len(source_contract_widget.options) - 1
else:
  count = len(source_contract_widget.value)
print("Ready to copy %d contract(s) to %d %s" % (count, len(destination_entities), "entity" if len(destination_entities) == 1 else "entities"))

In [ ]:
#@markdown Running copy-functions for each destination entity:
if source_contract_widget.value[0] == '':
  contract_list = source_contract_widget.options[1:]
  contracts = []
  for x in contract_list:
    contracts.append(x[1])
  contract_list = contracts
else:
  contract_list = source_contract_widget.value
for i,document_id in enumerate(contract_list):
  document_data = get_document_data(contract_result, document_id)
  print("Copy contract: \"" + document_data["name"] + "\"")
  source_document = get_source_document(document_data, direct=True)
  for dest in destination_entities:
    print("- Destination entity: \"" + dest["name"] + "\":")
    # Get or create destination template
    new_doc_id = upload_destination_document(source_document, document_data, dest)

print("All contracts has been copied")